In [1]:
import pandas as pd

from datetime import date 
from datetime import datetime, timedelta
import sys

import os
from pathlib import Path

import time

import re

import shutil

from os import listdir
from os.path import isfile, join

In [2]:
import sys 
#!{sys.executable} -m pip install --user matplotlib

# **Make comp dfs**

**Read sites list with ratings**

In [3]:
sites_list_df = pd.read_csv('iq_sites_list_rated(updated).csv')
sites_list_df.head(1)

,news_site,url,rate,Unnamed: 3
0,creteplus.gr,https://creteplus.gr,high,NaN


**Define column names**

In [4]:
header_names = ["type_s", # τύπος social
                "url_s", # url του social
                "external_link_s", # url του άρθρου
                "source_s", # όνομα του social
                "timestamp_dt", # ημερομηνία δημοσίευσης στο social
                "text_txt_el", # κείμενο στο social
                "retweet_count_l", # πλήθος retweets (twitter)
                "followers_i", # πλήθος followers (twitter)
                "following_i", # πλήθος following (twitter)
                "mentions_i", # πλήθος αναφορών (twitter)
                "comments_i", # πλήθος σχολίων (fb)
                "likes_i", # πλήθος likes (fb
                "full_artcl_text" # κείμενο άρθρου στο site (δεν περιλαμβάνεται ο τίτλος)
               ]

**Make comp dfs for high and low quality sites**

In [ ]:
# create dictionaries for the high and low sites dfs
dfs_high = {}
dfs_low = {}

# path to csvs from innews
main_csvs_path = '0_innews_data_sets/iqcsv/'

# read csv, make df, put it in the dictionary
for index, row in sites_list_df.iterrows():
    key = row['news_site']
    csv_name = key + '.csv'
    site_df = pd.read_csv(main_csvs_path + csv_name, header=None, names=header_names, on_bad_lines='skip')
    site_df['site'] = key
    if row['rate'] == 'high':
        dfs_high[key] = site_df
    elif row['rate'] == 'low':
        dfs_low[key] = site_df

In [ ]:
# make comp dfs for high and low dfs
for q in ['high', 'low']:
    if q == 'high':
        for i in range(0, len(dfs_high)-1):
            if i == 0:
                high_comp_df = pd.concat([dfs_high[list(dfs_high.keys())[i]], dfs_high[list(dfs_high.keys())[i+1]]])
            else:
                high_comp_df = pd.concat([high_comp_df, dfs_high[list(dfs_high.keys())[i+1]]])
    elif q == 'low':
        for i in range(0, len(dfs_low)-1):
            if i==0:
                low_comp_df = pd.concat([dfs_low[list(dfs_low.keys())[i]], dfs_low[list(dfs_low.keys())[i+1]]])
            else:
                low_comp_df = pd.concat([low_comp_df, dfs_low[list(dfs_low.keys())[i+1]]])
high_comp_df.reset_index(drop=True, inplace=True)
low_comp_df.reset_index(drop=True, inplace=True)

**Add an ID column**

In [ ]:
high_comp_df.insert(0, 'in_id', high_comp_df.index)
low_comp_df.insert(0, 'in_id', low_comp_df.index)

### **Initial observations**

In [ ]:
print('start unique article urls high:', len(high_comp_df.external_link_s.unique()))
print('start unique article urls low:', len(low_comp_df.external_link_s.unique()))
print('start unique article text high:', len(high_comp_df.full_artcl_text.unique()))
print('start unique article text low:', len(low_comp_df.full_artcl_text.unique()))
print('start unique post url high:', len(high_comp_df.url_s.unique()))
print('start unique post url low:', len(low_comp_df.url_s.unique()))
print('start unique post text high:', len(high_comp_df.text_txt_el.unique()))
print('start unique post text low:', len(low_comp_df.text_txt_el.unique()))

**Drop lines where typs_s is not twitter or facebook (NEW dfs)**

In [ ]:
print(len(high_comp_df))
print(len(low_comp_df))

In [ ]:
print(len(high_comp_df[(high_comp_df.type_s != 'twitter') & (high_comp_df.type_s != 'facebook')]))
print(len(low_comp_df[(low_comp_df.type_s != 'twitter') & (low_comp_df.type_s != 'facebook')]))

In [ ]:
high_comp_df2 = high_comp_df.drop(high_comp_df[(high_comp_df.type_s != 'twitter') & (high_comp_df.type_s != 'facebook')].index)
low_comp_df2 = low_comp_df.drop(low_comp_df[(low_comp_df.type_s != 'twitter') & (low_comp_df.type_s != 'facebook')].index)

In [ ]:
print(high_comp_df2.shape)
print(low_comp_df2.shape)

**Drop lines if the site is not greek (NEW dfs)**

In [ ]:
# make lists with NOT greek sites names
not_greek_sites_high = high_comp_df2[high_comp_df2.apply(lambda x: False if x ['site'].endswith('.gr')
                                  else True, axis=1)]['site'].unique()
not_greek_sites_low = low_comp_df2[low_comp_df2.apply(lambda x: False if x ['site'].endswith('.gr')
                                  else True, axis=1)]['site'].unique()
print('nbr of not_greek_sites_high:', len(not_greek_sites_high))
print(list(not_greek_sites_high))
print('nbr of not_greek_sites_low:', len(not_greek_sites_low))
print(list(not_greek_sites_low))

In [ ]:
false_not_greek_sites_high = ['newsvolos.com', 'myvolos.net', 'luben.tv', 'xenesglosses.eu', 'paok24.com',
                              'aek365.com', 'athens.indymedia.org', 'politisonline.com']
false_not_greek_sites_low = ['national-pride.org', 'hellasjournal.com', 'tilestwra.com',
                             'planet-greece.blogspot.com', 'enallaktikidrasi.com', 'press-gr.com',
                             'inewsgr.com', 'oroskopos.tv', 'thesecretrealtruth.blogspot.com',
                             'mignatiou.com', 'treloi.eu', 'touristikosodigos.com', 'i-eidisi.com',
                             'ant1iwo.com', 'patrisnews.com', 'apotis4stis5.com', 'philenews.com']

In [ ]:
# make lists with greek sites names
greek_sites_high = high_comp_df2[high_comp_df2.apply(lambda x: True if x ['site'].endswith('.gr')
                                  else False, axis=1)]['site'].unique()
greek_sites_low = low_comp_df2[low_comp_df2.apply(lambda x: True if x ['site'].endswith('.gr')
                                  else False, axis=1)]['site'].unique()

In [ ]:
# add false_not_greek_sites to lists with greek sites names
greek_sites_high_new = list(greek_sites_high) + false_not_greek_sites_high
greek_sites_low_new = list(greek_sites_low) + false_not_greek_sites_low

In [ ]:
high_comp_df3 = high_comp_df2[high_comp_df2['site'].isin(greek_sites_high_new)]
low_comp_df3 = low_comp_df2[low_comp_df2['site'].isin(greek_sites_low_new)]

In [ ]:
print(len(high_comp_df3))
print(len(low_comp_df3))

In [ ]:
# remove false_not_greek_sites from not_greek_sites list
not_greek_sites_high_new = []
for site in list(not_greek_sites_high):
    if site not in false_not_greek_sites_high:
        not_greek_sites_high_new.append(site)
not_greek_sites_low_new = []
for site in list(not_greek_sites_low):
    if site not in false_not_greek_sites_low:
        not_greek_sites_low_new.append(site)
print('nbr of not_greek_sites_high:', len(not_greek_sites_high_new))
print(not_greek_sites_high_new)
print('nbr of not_greek_sites_low:', len(not_greek_sites_low_new))
print(not_greek_sites_low_new)

**Drop lines whith NaN values in "full_artcl_text"**

In [ ]:
print('null full_article_text high:', high_comp_df3[high_comp_df3['full_artcl_text'].isnull()].in_id.count())
print('null full_article_text low:', low_comp_df3[low_comp_df3['full_artcl_text'].isnull()].in_id.count())

In [ ]:
print(len(high_comp_df3[high_comp_df3['full_artcl_text'].isnull()].external_link_s.unique()))
print(len(low_comp_df3[low_comp_df3['full_artcl_text'].isnull()].external_link_s.unique()))

In [ ]:
high_comp_df4 = high_comp_df3.drop(high_comp_df3[high_comp_df3['full_artcl_text'].isnull()].index)#.reset_index(drop=True)
low_comp_df4 = low_comp_df3.drop(low_comp_df3[low_comp_df3['full_artcl_text'].isnull()].index)#.reset_index(drop=True)
print(len(high_comp_df4))
print(len(low_comp_df4))

In [ ]:
high_comp_df4.columns

### **Corpus description**

In [ ]:
print('unique article urls high:', len(high_comp_df4.external_link_s.unique()))
print('unique article urls low:', len(low_comp_df4.external_link_s.unique()))

In [ ]:
print('unique full article text high:', len(high_comp_df4.full_artcl_text.unique()))
print('unique sites low:', len(low_comp_df4.full_artcl_text.unique()))

In [ ]:
print('unique sites high:', len(high_comp_df4.site.unique()))
print('unique sites low:', len(low_comp_df4.site.unique()))

In [ ]:
high_comp_df4.isnull().T.any().T.sum()

In [ ]:
high_comp_df[high_comp_df.isnull().T.any()]

In [ ]:
high_comp_df4.groupby('external_link_s').size().sort_values()

In [ ]:
high_comp_df4.groupby('type_s')[['in_id']].count()

In [ ]:
high_comp_df4.groupby(['external_link_s','type_s'], as_index=False)['in_id'].count()

In [ ]:
df_out = df.copy()
df_out = (df_out.groupby(['cluster', 'merchant']).agg({'latitude' : 'first',
                                                  'longitude' : 'first',
                                                  'city' : 'first',
                                                  'category' : 'first',
                                                  'cluster' : 'count'})
          .rename({'cluster' : 'cluster_count'},axis=1).reset_index()
          .sort_values(['cluster', 'cluster_count'], ascending = [True, False])
          .drop('cluster_count', axis=1))
df_out

**Check if source_s == site**

In [ ]:
len(high_comp_df4[(high_comp_df4.type_s == 'twitter') & (high_comp_df4.source_s == high_comp_df4.site)])

In [ ]:
len(high_comp_df4[(high_comp_df4.type_s == 'facebook') & (high_comp_df4.source_s == high_comp_df4.site)])

In [ ]:
high_comp_df4[(high_comp_df4.type_s == 'facebook') & (high_comp_df4.source_s == high_comp_df4.site)
             ].groupby('site')['site'].count()

### **Important Notes:** 
- Δεν υπάρχει κανένα tweet στο οποίο το source_s == site
- Yπάρχει μόνο στο facebook και αφορά μόνο 8 sites

Αυτό συνιστά τεράστιο πρόβλημα γιατί χάνεται πάρα πολύ πληροφορία από το dataset

**Ποια source_s μοιάζουν πολύ με το site?**

**High quality**

In [ ]:
# high sites that are likely to be the social source
sitesL_with_social_high = high_comp_df4[high_comp_df4.apply(lambda x: True if x ['source_s'].lower()
                                  .startswith(x['site'][:-3]) else False, axis=1)]['site'].unique()
sitesL_with_social_high

In [ ]:
# high source_s that are likely to belong to sites
sourceL_with_social_high = high_comp_df4[high_comp_df4.apply(lambda x: True if x ['source_s'].lower()
                                  .startswith(x['site'][:-3]) else False, axis=1)]['source_s'].unique()
sourceL_with_social_high

In [ ]:
print(len(sitesL_with_social_high))
print(len(sourceL_with_social_high))

**Low quality**

In [ ]:
# low sites that are likely to be the social source
sitesL_with_social_low = low_comp_df4[low_comp_df4.apply(lambda x: True if x ['source_s'].lower()
                                  .startswith(x['site'][:-3]) else False, axis=1)]['site'].unique()
sitesL_with_social_low

In [ ]:
# low source_s that are likely to belong to sites
sourceL_with_social_low = low_comp_df4[low_comp_df4.apply(lambda x: True if x ['source_s'].lower()
                                  .startswith(x['site'][:-3]) else False, axis=1)]['source_s'].unique()
sourceL_with_social_low

In [ ]:
print(len(sitesL_with_social_low))
print(len(sourceL_with_social_low))

**Create new source_s column**

**High quality**

In [ ]:
high_comp_df5 = high_comp_df4
low_comp_df5 = low_comp_df4

In [ ]:
# make a 'new_source_s' column with True values if 'source_s' matches 'site'
high_comp_df5['new_source_s'] = high_comp_df5.apply(lambda x: True if x ['source_s'].lower()
                                  .startswith(x['site'][:-3]) else False, axis=1)
high_comp_df5['source_new'] = high_comp_df5.apply(lambda x: x['site']
                                                  if x['new_source_s']==True else pd.NA, axis=1)
high_comp_df5.head(1)

In [ ]:
# drop lines where source_s doesn't match site
high_comp_df5 = high_comp_df5.dropna(subset=['source_new'])
print(len(high_comp_df5))
print(len(high_comp_df5.source_new.unique()))

**Low quality**

In [ ]:
# make a 'new_source_s' column with True values if 'source_s' matches 'site'
low_comp_df5['new_source_s'] = low_comp_df5.apply(lambda x: True if x ['source_s'].lower()
                                  .startswith(x['site'][:-3]) else False, axis=1)
low_comp_df5['source_new'] = low_comp_df5.apply(lambda x: x['site']
                                                  if x['new_source_s']==True else pd.NA, axis=1)
low_comp_df5.head(1)

In [ ]:
# drop lines where source_s doesn't match site
low_comp_df5 = low_comp_df5.dropna(subset=['source_new'])
print(len(low_comp_df5))
print(len(low_comp_df5.source_new.unique()))

### **Datasets report**

In [ ]:
print('DATASETS REPORT')
print('+'*15)
print('\n')
print('start datasets')
print('='*14)
print('start nbr of rows high:', len(high_comp_df))
print('start nbr of rows low:', len(low_comp_df))
print('-'*10)
print('start nbr of articles high:', len(high_comp_df.external_link_s.unique()))
print('start nbr of articles low:', len(low_comp_df.external_link_s.unique()))
print('-'*10)
print('start nbr of sites high:', len(high_comp_df.site.unique()))
print('start nbr of sites low:', len(low_comp_df.site.unique()))
print('-'*10)
print('\n')
print('final datasets (greek sites only, social posts from sites with identical name, full_artcl_text is not NaN)')
print('='*10)
print('final nbr of rows high:', len(high_comp_df5))
print('final nbr of rows low:', len(low_comp_df5))
print('-'*10)
print('final nbr of articles high:', len(high_comp_df5.external_link_s.unique()))
print('final nbr of articles low:', len(low_comp_df5.external_link_s.unique()))
print('-'*10)
print('final nbr of sites high:', len(high_comp_df5.site.unique()))
print('final nbr of sites low:', len(low_comp_df5.site.unique()))
print('-'*10)

## **Construct dfs**

### **High quality**

In [ ]:
high_comp_df5.head(1)

**Create high_comp_df for facebook**

In [ ]:
high_comp_df_fb = high_comp_df5[high_comp_df5.type_s == 'facebook'][['in_id', 'site', 'external_link_s',
                                                                     'full_artcl_text', 'url_s', 'text_txt_el',
                                                                    'likes_i', 'comments_i', 'timestamp_dt']]
high_comp_df_fb.head(1)

**Create high_comp_df for twitter**

In [ ]:
high_comp_df_tw = high_comp_df5[high_comp_df5.type_s == 'twitter'][['in_id', 'external_link_s', 'full_artcl_text',
                                                                     'url_s', 'text_txt_el', 'retweet_count_l',
                                                                    'followers_i', 'following_i', 'mentions_i',
                                                                    'timestamp_dt']]
high_comp_df_tw.head(1)

In [ ]:
print(len(high_comp_df_fb))
print(len(high_comp_df_tw))

**Merge fb and tw (on article_url and full_text)**

In [ ]:
high_comp_merged_df = pd.merge(high_comp_df_fb, high_comp_df_tw, on=["external_link_s", "full_artcl_text"])
high_comp_merged_df

In [ ]:
high_comp_merged_df.drop_duplicates(subset="external_link_s", keep='first', inplace=False)

### **Low quality**

In [ ]:
low_comp_df5.head(1)

**Create low_comp_df for facebook**

In [ ]:
low_comp_df_fb = low_comp_df5[low_comp_df5.type_s == 'facebook'][['in_id', 'site', 'external_link_s',
                                                                     'full_artcl_text', 'url_s', 'text_txt_el',
                                                                    'likes_i', 'comments_i', 'timestamp_dt']]
low_comp_df_fb.head(1)

**Create low_comp_df for twitter**

In [ ]:
low_comp_df_tw = low_comp_df5[low_comp_df5.type_s == 'twitter'][['in_id', 'external_link_s', 'full_artcl_text',
                                                                     'url_s', 'text_txt_el', 'retweet_count_l',
                                                                    'followers_i', 'following_i', 'mentions_i',
                                                                    'timestamp_dt']]
low_comp_df_tw.head(1)

In [ ]:
print(len(low_comp_df_fb))
print(len(low_comp_df_tw))

**Merge fb and tw (on article_url and full_text)**

In [ ]:
low_comp_merged_df = pd.merge(low_comp_df_fb, low_comp_df_tw, on=["external_link_s", "full_artcl_text"])
low_comp_merged_df

In [ ]:
low_comp_merged_df.drop_duplicates(subset="external_link_s", keep='first', inplace=False).reset_index()